# Assignment 5 - Text Analysis
An explanation this assignment could be found in the .pdf explanation document


## Materials to review for this assignment
<h4>From Moodle:</h4> 
<h5><u>Review the notebooks regarding the following python topics</u>:</h5>
<div class="alert alert-info">
&#x2714; <b>Working with strings</b> (tutorial notebook)<br/>
&#x2714; <b>Text Analysis</b> (tutorial notebook)<br/>
&#x2714; <b>Hebrew text analysis tools (tokenizer, wordnet)</b> (moodle example)<br/>
&#x2714; <b>(brief review) All previous notebooks</b><br/>
</div> 
<h5><u>Review the presentations regarding the following topics</u>:</h5>
<div class="alert alert-info">
&#x2714; <b>Text Analysis</b> (lecture presentation)<br/>
&#x2714; <b>(brief review) All other presentations</b><br/>
</div>

## Preceding Step - import modules (packages)
This step is necessary in order to use external modules (packages). <br/>

In [1]:
# --------------------------------------
import pandas as pd
import numpy as np
# --------------------------------------


# --------------------------------------
# ------------- visualizations:
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
# --------------------------------------


# ---------------------------------------
import sklearn
from sklearn import preprocessing, metrics, pipeline, model_selection, feature_extraction 
from sklearn import naive_bayes, linear_model, svm, neural_network, neighbors, tree
from sklearn import decomposition, cluster

from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV 
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.metrics import mean_squared_error, r2_score, silhouette_score
from sklearn.preprocessing import MinMaxScaler, StandardScaler, LabelEncoder

from sklearn.svm import LinearSVC
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import Perceptron, SGDClassifier
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
# ---------------------------------------


# ----------------- output and visualizations: 
import warnings
from sklearn.exceptions import ConvergenceWarning
warnings.simplefilter("ignore")
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter("ignore", category=ConvergenceWarning)
# show several prints in one cell. This will allow us to condence every trick in one cell.
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
%matplotlib inline
pd.pandas.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.3f' % x)
# ---------------------------------------

### Text analysis and String manipulation imports:

In [2]:
# --------------------------------------
# --------- Text analysis and Hebrew text analysis imports:
# vectorizers:
from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

# regular expressions:
import re
# --------------------------------------

### (optional) Hebrew text analysis - WordNet (for Hebrew)
Note: the WordNet is not a must

#### (optional) Only if you didn't install Wordnet (for Hebrew) use:

In [3]:
# word net installation:

# unmark if you want to use and need to install
# !pip install wn
# !python -m wn download omw-he:1.4

In [4]:
# word net import:

# unmark if you want to use:
# import wn

### (optional) Hebrew text analysis - hebrew_tokenizer (Tokenizer for Hebrew)
Note: the hebrew_tokenizer is not a must

#### (optional) Only if you didn't install hebrew_tokenizer use:

In [5]:
# Hebrew tokenizer installation:

# unmark if you want to use and need to install:
# !pip install hebrew_tokenizer

In [6]:
# Hebrew tokenizer import:

# unmark if you want to use:
# import hebrew_tokenizer as ht

### Reading input files
Reading input files for train annotated corpus (raw text data) corpus and for the test corpus

In [7]:
train_filename = 'annotated_corpus_for_train.csv'
test_filename  = 'corpus_for_test.csv'
df_train = pd.read_csv(train_filename, index_col=None, encoding='utf-8')
df_test  = pd.read_csv(test_filename, index_col=None, encoding='utf-8')

In [8]:
df_train.head(8)
df_train.shape

,story,gender
0,"כשחבר הזמין אותי לחול, לא באמת חשבתי שזה יקרה,...",m
1,לפני שהתגייסתי לצבא עשיתי כל מני מיונים ליחידו...,m
2,מאז שהתחילו הלימודים חלומו של כל סטודנט זה הפנ...,f
3,"כשהייתי ילד, מטוסים היה הדבר שהכי ריתק אותי. ב...",m
4,‏הייתי מדריכה בכפר נוער ומתאם הכפר היינו צריכי...,f
5,לפני כ3 חודשים טסתי לרומא למשך שבוע. טסתי במטו...,f
6,אני כבר שנתיים נשוי והשנה אני ואישתי סוף סוף י...,m
7,השנה התחלנו שיפוץ בדירה שלנו בתל אביב. הדירה ה...,f


(753, 2)

In [9]:
df_test.head(3)
df_test.shape

,test_example_id,story
0,0,כל קיץ אני והמשפחה נוסעים לארצות הברית לוס אנג...
1,1,"הגעתי לשירות המדינה אחרי שנתיים כפעיל בתנועת ""..."
2,2,אחת האהבות הגדולות שלי אלו הכלבים שלי ושל אישת...


(323, 2)

### Your implementation:
Write your code solution in the following code-cells

In [10]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import make_scorer
from sklearn.base import clone

To build the best model to predict the writers' gender in a dataset, the dataset has to go through several stages of preparation: 
- Preprocessing - cleaning the data from unnecessary characters.
- Vectorization - converting the texts to feature vectors.
- Evaluation - evaluation the average f1 score for female and male, and the f1 scores for the genders separately.
- Prediction - using the model to predict the test set

## First Step - Preprocessing
To keep all the Hebrew characters, I created a function that removes any character that is not a Hebrew letter.

In [11]:
# YOUR CODE HERE
def clean_text(df_train):
    df_copy = df_train.copy()
    stories_lst = list(df_copy["story"])
    for index, txt in enumerate(stories_lst):
        tmp_txt = ""
        for c in txt:
            if (c >= 'א' and c <= 'ת') or c == ' ':
                tmp_txt += c

        stories_lst[index] = tmp_txt
    df_copy["story"] = stories_lst
    return df_copy

## Second Step - Vectorization
In the vectorization process of text features we need to convert text to a set of representative numerical values so i used TfidfVectorizer and CountVectorizer.

In [12]:
# YOUR CODE HERE
def tfidf_vectorization(min_df, max_df, ngrams, train_df):
    vec = TfidfVectorizer(max_df=max_df, min_df=min_df, ngram_range=ngrams)
    X_train = vec.fit_transform(train_df["story"])
    X_train_df = pd.DataFrame(X_train.toarray(), columns=vec.get_feature_names_out())
    X_train_df["gender"] = df_train["gender"]
    return vec, X_train_df

def count_vectorization(min_df, max_df, ngrams, train_df):
    vec = CountVectorizer(max_df=max_df, min_df=min_df, ngram_range=ngrams)
    X_train = vec.fit_transform(train_df["story"])
    X_train_df = pd.DataFrame(X_train.toarray(), columns=vec.get_feature_names_out())
    X_train_df["gender"] = df_train["gender"]
    return vec, X_train_df

## Third Step - Evaluation
The evaluation metric is measured by f1 score, I evaluate the average f1 score by calculating the f1 score for female and the f1 score for male.
To get the f1 score for each gender separately, I duplicate the target column, the "gender" column. In one of the duplicates, the "f" is labled 0 and "m" is 1, in the other the "f" is labaled 1 and "m" is 0.

In [13]:
def female_male_split_data(X_train_df):
    X_train_male = X_train_df.copy()
    X_train_female = X_train_df.copy()

    X_train_male["gender"] = np.where(X_train_male["gender"] == 'f', 0, 1)
    X_train_female["gender"] = np.where(X_train_female["gender"] == 'f', 1, 0)

    target_columns = X_train_df.columns[X_train_df.columns != "gender"]
    X_train = X_train_df[target_columns]

    y_male = X_train_male["gender"]
    y_female = X_train_female["gender"]
    return X_train, y_male, y_female

Now that the data is splitted, I get the best params for the model with GridSearchCV, and get the f1 score for each gender with cross_val_score; this calculate the f1 average.

In [14]:
# YOUR CODE HERE
def f1_calculation(model, param, X_train, y_male, y_female):
    clf = GridSearchCV(model, param, scoring=make_scorer(f1_score), cv=5, n_jobs=-1)
    clf.fit(X_train, y_male)
    best_params_male = clf.best_params_

    clf.fit(X_train, y_female)
    best_params_female = clf.best_params_

    model.set_params(**best_params_male)
    f1_score_male = cross_val_score(model, X_train, y_male, scoring=make_scorer(f1_score), cv=5).mean()

    model.set_params(**best_params_female)
    f1_score_female = cross_val_score(model, X_train, y_female, scoring=make_scorer(f1_score), cv=5).mean()

    f1_average = (f1_score_male + f1_score_female) / 2
    
    return f1_score_male, f1_score_female, f1_average,model, best_params_male

## Forth Step - Prediction
The last step is to activate the best model on the test set, df_test. First, the test set go through the same preprocessing as the train set to clean it's data, and then the model can predict the genders.

In [15]:
def predict_test_df(model,vectorizer,df_test,X_train,y_train,best_params):
    full_txt = clean_text(df_test)
    
    df_test_vec = vectorizer.transform(full_txt["story"])
    X_test_df = pd.DataFrame(df_test_vec.toarray(), columns=vectorizer.get_feature_names_out())

    model.set_params(**best_params)
    clf = model.fit(X_train,y_train)
    y_pred = clf.predict(X_test_df)
    
    y_pred = np.where(y_pred == 1,'m','f')
    pred_df = df_test.copy()
    pred_df["gender"] = y_pred
    return pred_df
    

# Main
### Setting the Models
I chose three models and their best params to get the best result to train and predict. The models are KNeighborsClassifier, MultinomialNB and LogisticRegression, where MultinomialNB is a probabilistic model used for multi-class classification that assumes features have discrete multinomial distributions and calculates class probabilities using Bayes' theorem, and LogisticRegression is a statistical model used for binary classification that estimates the probability of an outcome based on input features using a logistic function.

In [16]:
# Add as many code cells as you need
# YOUR CODE HERE
grid_params = {
        'grid_param_logistic': {
        'penalty': ['l1', 'l2'],
        'C': [0.01, 0.1, 1.0, 10.0, 100.0],
        'solver': ['lbfgs', 'newton-cg', 'liblinear'],
        'class_weight': [{0: 1, 1: 1}, {0: 1, 1: 3}, {0: 1, 1: 5}],
        'random_state': [41, 42],
        'max_iter': [1000, 2000]
    },

    'param_grid_NB' : {
        'alpha': [0.01, 0.1, 1.0, 10.0, 100.0],
        'fit_prior': [True, False],
        'class_prior': [None, [0.2, 0.8], [0.3, 0.7], [0.4, 0.6], [0.5, 0.5], [0.6, 0.4], [0.7, 0.3], [0.8, 0.2], [0.9, 0.1]]
    },

    'param_grid_KNN': {'n_neighbors':range(1,25,2)}}

models = {'LogisticRegression': LogisticRegression(),
          'MultinomialNB': MultinomialNB(),
          'KNeighborsClassifier': KNeighborsClassifier()}

f1_results = []


### Calculating The Models' F1 Scores
Using the functions above to calculate the models' f1 score average with CountVectorizer and TfidfVectorizer.

In [17]:
print("CountVectorizer: \n")
full_data = clean_text(df_train.copy())
vec_model, vec_cv_df = count_vectorization(5, 1.0, (1,3), full_data.copy())

X_train, y_male, y_female = female_male_split_data(vec_cv_df.copy())

f1_score_male, f1_score_female, f1_average, model, best_params_male = f1_calculation(clone(models['LogisticRegression']), grid_params['grid_param_logistic'], X_train.copy(), y_male, y_female)
print(models['LogisticRegression'])
print("\nThe Used: ", model, "\n")
print("f1 male: ", f1_score_male)
print("f1 female: ", f1_score_female)
print("f1 average: ", f1_average)
f1_results.append({'vectorizer':vec_model,'model':clone(models['LogisticRegression']), 'result': f1_average, 'best_params_male': best_params_male})

f1_score_male, f1_score_female, f1_average, model, best_params_male = f1_calculation(clone(models['MultinomialNB']), grid_params['param_grid_NB'], X_train.copy(), y_male, y_female)
print(models['MultinomialNB'])
print("\nThe Used: ", model, "\n")
print("f1 male: ", f1_score_male)
print("f1 female: ", f1_score_female)
print("f1 average: ", f1_average)
f1_results.append({'vectorizer':vec_model,'model':clone(models['MultinomialNB']), 'result': f1_average,'best_params_male': best_params_male,})

f1_score_male, f1_score_female, f1_average, model, best_params_male = f1_calculation(clone(models['KNeighborsClassifier']), grid_params['param_grid_KNN'], X_train.copy(), y_male, y_female)
print(models['KNeighborsClassifier'])
print("\nThe Used: ", model, "\n")
print("f1 male: ", f1_score_male)
print("f1 female: ", f1_score_female)
print("f1 average: ", f1_average)
f1_results.append({'vectorizer':vec_model,'model':clone(models['KNeighborsClassifier']), 'result': f1_average,'best_params_male': best_params_male})

CountVectorizer: 

LogisticRegression()

The Used:  LogisticRegression(C=0.01, class_weight={0: 1, 1: 5}, max_iter=1000,
                   random_state=41, solver='liblinear') 

f1 male:  0.8866496675900954
f1 female:  0.5745444047277307
f1 average:  0.7305970361589131
MultinomialNB()

The Used:  MultinomialNB(class_prior=[0.2, 0.8]) 

f1 male:  0.886145134030806
f1 female:  0.5392957433048704
f1 average:  0.7127204386678382
KNeighborsClassifier()

The Used:  KNeighborsClassifier(n_neighbors=1) 

f1 male:  0.8666243223840044
f1 female:  0.26671159029649594
f1 average:  0.5666679563402501


In [18]:
print("TfidfVectorizer: \n")
full_data = clean_text(df_train.copy())
vec_model, vec_tfidf_df = tfidf_vectorization(5, 1.0, (1,3), full_data.copy())

X_train, y_male, y_female = female_male_split_data(vec_tfidf_df.copy())

f1_score_male, f1_score_female, f1_average, model, best_params_male = f1_calculation(clone(models['LogisticRegression']), grid_params['grid_param_logistic'], X_train.copy(), y_male, y_female)
print(models['LogisticRegression'])
print("\nThe Used: ", model, "\n")
print("f1 male: ", f1_score_male)
print("f1 female: ", f1_score_female)
print("f1 average: ", f1_average)
f1_results.append({'vectorizer':vec_model,'model':clone(models['LogisticRegression']), 'result': f1_average, 'best_params_male': best_params_male})

f1_score_male, f1_score_female, f1_average, model, best_params_male = f1_calculation(clone(models['MultinomialNB']), grid_params['param_grid_NB'], X_train.copy(), y_male, y_female)
print(models['MultinomialNB'])
print("\nThe Used: ", model, "\n")
print("f1 male: ", f1_score_male)
print("f1 female: ", f1_score_female)
print("f1 average: ", f1_average)
f1_results.append({'vectorizer':vec_model,'model':clone(models['MultinomialNB']), 'result': f1_average,'best_params_male': best_params_male})

f1_score_male, f1_score_female, f1_average, model, best_params_male = f1_calculation(clone(models['KNeighborsClassifier']), grid_params['param_grid_KNN'], X_train.copy(), y_male, y_female)
print(models['KNeighborsClassifier'])
print("\nThe Used: ", model, "\n")
print("f1 male: ", f1_score_male)
print("f1 female: ", f1_score_female)
print("f1 average: ", f1_average)
f1_results.append({'vectorizer':vec_model,'model':clone(models['KNeighborsClassifier']), 'result': f1_average,'best_params_male': best_params_male})


TfidfVectorizer: 

LogisticRegression()

The Used:  LogisticRegression(class_weight={0: 1, 1: 5}, max_iter=1000, random_state=41,
                   solver='liblinear') 

f1 male:  0.8898893099752037
f1 female:  0.5917145518937821
f1 average:  0.7408019309344929
MultinomialNB()

The Used:  MultinomialNB(alpha=0.1, class_prior=[0.4, 0.6]) 

f1 male:  0.8857013607299683
f1 female:  0.5773899388394426
f1 average:  0.7315456497847055
KNeighborsClassifier()

The Used:  KNeighborsClassifier(n_neighbors=1) 

f1 male:  0.8682678317349819
f1 female:  0.44637724832846787
f1 average:  0.6573225400317249


##### show the list of results

In [19]:
f1_results

[{'vectorizer': CountVectorizer(min_df=5, ngram_range=(1, 3)),
  'model': LogisticRegression(),
  'result': 0.7305970361589131,
  'best_params_male': {'C': 100.0,
   'class_weight': {0: 1, 1: 5},
   'max_iter': 1000,
   'penalty': 'l1',
   'random_state': 41,
   'solver': 'liblinear'}},
 {'vectorizer': CountVectorizer(min_df=5, ngram_range=(1, 3)),
  'model': MultinomialNB(),
  'result': 0.7127204386678382,
  'best_params_male': {'alpha': 0.1,
   'class_prior': [0.9, 0.1],
   'fit_prior': True}},
 {'vectorizer': CountVectorizer(min_df=5, ngram_range=(1, 3)),
  'model': KNeighborsClassifier(),
  'result': 0.5666679563402501,
  'best_params_male': {'n_neighbors': 17}},
 {'vectorizer': TfidfVectorizer(min_df=5, ngram_range=(1, 3)),
  'model': LogisticRegression(),
  'result': 0.7408019309344929,
  'best_params_male': {'C': 100.0,
   'class_weight': {0: 1, 1: 5},
   'max_iter': 1000,
   'penalty': 'l1',
   'random_state': 41,
   'solver': 'liblinear'}},
 {'vectorizer': TfidfVectorizer(min_

##### finding the best result from the list


In [20]:
max_result = {'vectorizer': "",'model':"", 'result': 0}
for result in f1_results:
    if result['result'] > max_result['result']:
        max_result = result
print(max_result)

{'vectorizer': TfidfVectorizer(min_df=5, ngram_range=(1, 3)), 'model': LogisticRegression(), 'result': 0.7408019309344929, 'best_params_male': {'C': 100.0, 'class_weight': {0: 1, 1: 5}, 'max_iter': 1000, 'penalty': 'l1', 'random_state': 41, 'solver': 'liblinear'}}


#### Predicting the gender
calling the prediction method on the test dataframe and returning the test dataframe with the predicted labels. at the end I'm presenting the 5 first and 5 last results.

In [21]:
pred_df = predict_test_df(max_result['model'],max_result['vectorizer'],df_test.copy(), X_train.copy(), y_male,max_result['best_params_male'])

In [22]:
pred_df.head()
pred_df.tail()

,test_example_id,story,gender
0,0,כל קיץ אני והמשפחה נוסעים לארצות הברית לוס אנג...,m
1,1,"הגעתי לשירות המדינה אחרי שנתיים כפעיל בתנועת ""...",m
2,2,אחת האהבות הגדולות שלי אלו הכלבים שלי ושל אישת...,m
3,3,"רגע הגיוס לצבא היה הרגע הכי משמעותי עבורי, אני...",m
4,4,אני הגעתי לברזיל ישר מקולומביה וגם אני עשיתי ע...,m


,test_example_id,story,gender
318,318,"בשנה האחרונה הרגשתי די תקוע בעבודה, השגרה הפכה...",m
319,319,אני ואילן חברים טובים מזה 20 שנה תמיד חלמנו לפ...,m
320,320,מידי יום שישי אני נוהג לנסוע בתחבורה ציבורית ס...,m
321,321,"לפני מספר חודשים, בשיא התחלואה של הגל השני, עמ...",m
322,322,היום בו דיווחתי על גניבה של האוטו שלי. בוקר אח...,m


In [23]:
pred_df["gender"].value_counts()

m    262
f     61
Name: gender, dtype: int64

### Save output to csv (optional)
After you're done save your output to the 'classification_results.csv' csv file.<br/>
We assume that the dataframe with your results contain the following columns:
* column 1 (left column): 'test_example_id'  - the same id associated to each of the test stories to be predicted.
* column 2 (right column): 'predicted_category' - the predicted gender value for each of the associated story. 

Assuming your predicted values are in the `df_predicted` dataframe, you should save you're results as following:

In [1]:
df_predicted.to_csv('classification_results.csv',index=False)